In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sentence_transformers import SentenceTransformer, util
import json, os, math
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
schema_root_path = "filtering_schema/src/schemas/coffeeshop-descriptions"
column_names = ['Table', 'Column', 'Description']
full_df = pd.DataFrame(columns=column_names)

for file in os.listdir(schema_root_path):
    schema_file_path = os.path.join(schema_root_path, file)
    with open(schema_file_path, 'r') as f:
        schema = json.load(f)

    temp_df = pd.DataFrame(schema['columns'].items() ,columns=['Column', 'Description'])
    temp_df['Table'] = schema['table']
    temp_df = temp_df[column_names]
    full_df = pd.concat([full_df, temp_df], ignore_index=True)

print(full_df.shape)
full_df

(19, 3)


,Table,Column,Description
0,happy_hour,HH_ID,Unique identifier for the happy hour event
1,happy_hour,Shop_ID,Identifier of the shop hosting the happy hour
2,happy_hour,Month,Month in which the happy hour takes place
3,happy_hour,Num_of_staff_in_charge,Number of staff members in charge during the h...
4,member,Member_ID,Unique identifier for the member
5,member,Name,Name of the member
6,member,Membership_card,Details of the membership card
7,member,Age,Age of the member
8,member,Time_of_purchase,Date and time of purchase
9,member,Level_of_membership,Level or type of membership


In [4]:
schema_classes_file_path = "src/spider/cofee_shop/coffee_shop_class.json"

with open(schema_classes_file_path, 'r') as f:
    schema_classes = json.load(f)
schema_classes

{'event_detail': 'This class contains columns related to event details, such as happy hour event identifiers, the shop hosting the event, the month in which the event takes place, and the number of staff members in charge during the event.',
 'member_information': 'This class contains columns related to member information, such as member identifiers, member names, membership card details, age of the member, time of purchase, level or type of membership, and address of the member.',
 'shop_information': 'This class contains columns related to shop information, such as shop identifiers, shop addresses, the number of staff members working at the shop, the score or rating of the shop, and the year when the shop opened.'}

In [5]:
sentenceemb_model = SentenceTransformer("filtering_schema/models/all-MiniLM-L6-v2")
for topic, info in schema_classes.items():
    schema_classes[topic] = sentenceemb_model.encode(info)

In [23]:
def most_relate_topic(text:str, topic_threshold_score:float=0.2, 
                      topic_select:bool=False, top_n:int=10, base_n:int=1):
    text_vec = sentenceemb_model.encode(text)
    topic_scores = [float(util.cos_sim(info, text_vec)) for info in schema_classes.values()]
    if topic_select:
        probs = (topic_scores / np.sum(topic_scores)) * top_n
        topic_selected = { key: max(base_n, math.ceil(score)) for key, score in zip(schema_classes.keys(), probs)}
        return topic_selected
    related_topic_indices = np.where(np.array(topic_scores) >= topic_threshold_score)[0]
    related_topics = [list(schema_classes.keys())[i] for i in related_topic_indices]
    return related_topics

In [24]:
full_df['Vector'] = full_df['Description'].apply(lambda x: sentenceemb_model.encode(x))
full_df['Topic'] = full_df['Description'].apply(most_relate_topic, args=(0.2,))
full_df.head()

,Table,Column,Description,Vector,Topic
0,happy_hour,HH_ID,Unique identifier for the happy hour event,"[-0.03035756, 0.11128711, 0.009328192, 0.03674...",[event_detail]
1,happy_hour,Shop_ID,Identifier of the shop hosting the happy hour,"[-0.020781243, 0.10354371, -0.022293467, 0.040...","[event_detail, shop_information]"
2,happy_hour,Month,Month in which the happy hour takes place,"[0.053571712, 0.09682498, -0.03012198, 0.02736...",[event_detail]
3,happy_hour,Num_of_staff_in_charge,Number of staff members in charge during the h...,"[0.048624337, 0.03614841, -0.005396093, 0.0233...",[event_detail]
4,member,Member_ID,Unique identifier for the member,"[-0.070519626, 0.01890726, -0.03272294, -0.004...","[event_detail, member_information, shop_inform..."


In [8]:
expanded_df = full_df.explode('Topic')
expanded_df.head()

,Table,Column,Description,Vector,Topic
0,happy_hour,HH_ID,Unique identifier for the happy hour event,"[-0.03035756, 0.11128711, 0.009328192, 0.03674...",event_detail
1,happy_hour,Shop_ID,Identifier of the shop hosting the happy hour,"[-0.020781243, 0.10354371, -0.022293467, 0.040...",event_detail
1,happy_hour,Shop_ID,Identifier of the shop hosting the happy hour,"[-0.020781243, 0.10354371, -0.022293467, 0.040...",shop_information
2,happy_hour,Month,Month in which the happy hour takes place,"[0.053571712, 0.09682498, -0.03012198, 0.02736...",event_detail
3,happy_hour,Num_of_staff_in_charge,Number of staff members in charge during the h...,"[0.048624337, 0.03614841, -0.005396093, 0.0233...",event_detail


In [9]:
expanded_df[expanded_df['Topic'] == 'shop_information'].index

Index([1, 8, 11, 12, 13, 14, 15], dtype='int64')

In [32]:
full_df['Table'].unique()

array(['happy_hour', 'member', 'shop', 'happy_hour_member'], dtype=object)

In [58]:
question = 'How many member paying each shop that has most happy hour'
def selected_columns(question, max_n=10) -> list:
    _df = expanded_df[['Table', 'Column', 'Vector', 'Topic']]
    _df['Score'] = _df['Vector'].apply(lambda x: float(util.cos_sim(x, sentenceemb_model.encode(question))))
    topic_selected = most_relate_topic(question, top_n=max_n, base_n=2, topic_select=True)
    used_schema = {table : dict() for table in full_df['Table'].unique()}
    used_cols = []
    for topic, num in topic_selected.items():
        selected_col_index = _df[_df['Topic'] == topic]['Score'].sort_values(ascending=False).head(num).index
        used_cols.extend(_df.loc[selected_col_index, 'Column'].to_list())
    used_cols = list(set(used_cols))
    for i, row in _df[_df['Column'].isin(used_cols)].iterrows():
        used_schema[row['Table']][row['Column']] = round(row['Score'],3)
        
    return used_schema

selected_columns(question)

{'happy_hour': {'Shop_ID': 0.716,
  'Month': 0.543,
  'Num_of_staff_in_charge': 0.746},
 'member': {'Member_ID': 0.162, 'Level_of_membership': 0.259},
 'shop': {'Shop_ID': 0.333, 'Num_of_staff': 0.57, 'Score': 0.4},
 'happy_hour_member': {'Member_ID': 0.579, 'Total_amount': 0.695}}

In [29]:
from filtering_schema.Description_base_linking import SchemaLinking
from transformers import AutoTokenizer, AutoModelForCausalLM

base_dir = "filtering_schema"
os.environ['nsql_model_path'] = os.path.join(base_dir, 'models', 'nsql-350M')
os.environ['sentence_emb_model_path'] = os.path.join(base_dir, 'models', 'all-MiniLM-L6-v2')
os.environ['schema_description_folder_path'] = os.path.join(base_dir, 'src', 'schemas', 'coffeeshop-descriptions')
os.environ['schema_data_types_folder_path'] = os.path.join(base_dir, 'src', 'schemas', 'coffeeshop-datatypes')
os.environ['column_threshold'] = '0.2'
os.environ['table_threshold'] = '0.2'
os.environ['max_select_column'] = '10'
os.environ['filter_table'] = 'False'
os.environ['verbose'] = 'False'


schema_link = SchemaLinking()
schema_link.selected_domain(schema_description_folder_path=os.environ.get('schema_description_folder_path'),
                            schema_data_types_folder_path=os.environ.get('schema_data_types_folder_path'))

tokenizer = AutoTokenizer.from_pretrained(os.environ.get('nsql_model_path'))
model = AutoModelForCausalLM.from_pretrained(os.environ.get('nsql_model_path'))
verbose = bool(os.environ.get('verbose').lower() == 'true')

In [59]:
schema_link.filter_schema(question)

Table string match  ----> member
Table string match  ----> shop


{'happy_hour': {'HH_ID': 0.489,
  'Shop_ID': 0.716,
  'Month': 0.543,
  'Num_of_staff_in_charge': 0.746},
 'happy_hour_member': {'HH_ID': 0.486,
  'Member_ID': 0.579,
  'Total_amount': 0.695},
 'member': {},
 'shop': {'Shop_ID': 0.333,
  'Num_of_staff': 0.57,
  'Score': 0.4,
  'Open_Year': 0.312}}

In [60]:
selected_columns(question)

{'happy_hour': {'Shop_ID': 0.716,
  'Month': 0.543,
  'Num_of_staff_in_charge': 0.746},
 'member': {'Member_ID': 0.162, 'Level_of_membership': 0.259},
 'shop': {'Shop_ID': 0.333, 'Num_of_staff': 0.57, 'Score': 0.4},
 'happy_hour_member': {'Member_ID': 0.579, 'Total_amount': 0.695}}

In [34]:

# TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf_vectorizer.fit_transform(full_df['Description'])

vectorizer = CountVectorizer(max_features=1000, stop_words='english')
dtm = vectorizer.fit_transform(full_df['Description'])

# Apply Latent Dirichlet Allocation (LDA)
n_topics = 3  # You can adjust the number of topics based on your needs
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
# topics = lda.fit_transform(tfidf_matrix)
topics = lda.fit_transform(dtm)

# Assign the most probable topic to each column
full_df['Topic'] = topics.argmax(axis=1)

grouped = full_df.groupby('Topic')
for group_name, group_data in grouped:
    print(f"Category: {group_name}")
    print(group_data[['Column', 'Description']])

Category: 0
                 Column                  Description
8      Time_of_purchase    Date and time of purchase
9   Level_of_membership  Level or type of membership
10              Address        Address of the member
Category: 1
             Column                                        Description
0             HH_ID         Unique identifier for the happy hour event
1           Shop_ID      Identifier of the shop hosting the happy hour
2             Month          Month in which the happy hour takes place
4         Member_ID                   Unique identifier for the member
5              Name                                 Name of the member
6   Membership_card                     Details of the membership card
7               Age                                  Age of the member
11          Shop_ID                     Unique identifier for the shop
16            HH_ID                 Identifier of the happy hour event
17        Member_ID  Identifier of the member particip